In [3]:
import pandas as pd
import numpy as np

In [4]:
teste = pd.read_csv("C:/Users/enzoj/Downloads/UNICAMP/GitHub/projects-of-data-science/Projetinho-aprendendo-a-usar-jupyter/Titanic/test.csv")
train = pd.read_csv("C:/Users/enzoj/Downloads/UNICAMP/GitHub/projects-of-data-science/Projetinho-aprendendo-a-usar-jupyter/Titanic/train.csv")

In [5]:
def transformar_sexo(valor):
    if valor == 'female':
        return 1
    else:
        return 0

Por que será que a nossa ultima predição foi tão ruim? Porque usamos apenas duas variaveis... e é bem mais complexo que isso.
Vamos usar a prática da Validação (treino). 

In [6]:
train['Sex_binario'] = train['Sex'].map(transformar_sexo)
teste['Sex_binario'] = teste['Sex'].map(transformar_sexo)

variaveis = ['Sex_binario', 'Age']

X = train[variaveis].fillna(-1)
y = train['Survived']

In [7]:
train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_binario
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [9]:
X_falso = np.arange(10)
X_falso

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [18]:
np.random.seed(1) #funcao capaz de controlar a aleatoriedade 
train_test_split(X_falso, test_size=0.5)

[array([3, 1, 7, 8, 5]), array([2, 9, 6, 4, 0])]

In [19]:
np.random.seed(1)
X_treino, X_valid, y_treino, y_valid = train_test_split(X, y, test_size=0.5)

In [20]:
X_treino.head()


,Sex_binario,Age
394,1,24.00
851,0,74.00
373,0,22.00
523,1,44.00
78,0,0.83


In [13]:
X_treino.shape, X_valid.shape, y_treino.shape, y_valid.shape


((445, 2), (446, 2), (445,), (446,))

In [14]:
modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
modelo.fit(X_treino, y_treino)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [21]:
p = modelo.predict(X_valid)

In [22]:
np.mean(y_valid == p)


0.7466367713004485

In [23]:
p = (X_valid['Sex_binario'] == 1).astype(np.int64)
np.mean(y_valid == p)

0.7623318385650224

Fazer um modo diferente para essa predição. Utilizamos um outro tipo de aleatoriedade, pois podemos ter um Outlier. Fazemos, então, a reamostragem. OU, então, a Validação Cruzada. 


In [24]:
X_falso

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [25]:
from sklearn.model_selection import KFold 

In [26]:
?KFold


In [27]:
kf = KFold(3, shuffle=True, random_state=0)
for linhas_treino, linhas_valid in kf.split(X_falso):
    print("Treino:", linhas_treino)
    print("Valid:", linhas_valid)
    print()

Treino: [0 1 3 5 6 7]
Valid: [2 4 8 9]

Treino: [0 2 3 4 5 8 9]
Valid: [1 6 7]

Treino: [1 2 4 6 7 8 9]
Valid: [0 3 5]



In [28]:
resultados = []
for rep in range(10):
    print("Rep:", rep)
    kf = KFold(5, shuffle=True, random_state=rep)
    
    for linhas_treino, linhas_valid in kf.split(X):
        print("Treino:", linhas_treino.shape[0])
        print("Valid:", linhas_valid.shape[0])

        X_treino, X_valid = X.iloc[linhas_treino], X.iloc[linhas_valid]
        y_treino, y_valid = y.iloc[linhas_treino], y.iloc[linhas_valid]

        modelo = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
        modelo.fit(X_treino, y_treino)

        p = modelo.predict(X_valid)

        acc = np.mean(y_valid == p)
        resultados.append(acc)
        print("Acc:", acc)
        print()
        #print(X_treino.head())
        #print()

Rep: 0
Treino: 712
Valid: 179
Acc: 0.7988826815642458

Treino: 713
Valid: 178
Acc: 0.7359550561797753

Treino: 713
Valid: 178
Acc: 0.7808988764044944

Treino: 713
Valid: 178
Acc: 0.797752808988764

Treino: 713
Valid: 178
Acc: 0.7808988764044944

Rep: 1
Treino: 712
Valid: 179
Acc: 0.7374301675977654

Treino: 713
Valid: 178
Acc: 0.7247191011235955

Treino: 713
Valid: 178
Acc: 0.7808988764044944

Treino: 713
Valid: 178
Acc: 0.7921348314606742

Treino: 713
Valid: 178
Acc: 0.7921348314606742

Rep: 2
Treino: 712
Valid: 179
Acc: 0.7653631284916201

Treino: 713
Valid: 178
Acc: 0.7865168539325843

Treino: 713
Valid: 178
Acc: 0.7865168539325843

Treino: 713
Valid: 178
Acc: 0.7808988764044944

Treino: 713
Valid: 178
Acc: 0.7640449438202247

Rep: 3
Treino: 712
Valid: 179
Acc: 0.7653631284916201

Treino: 713
Valid: 178
Acc: 0.7471910112359551

Treino: 713
Valid: 178
Acc: 0.7808988764044944

Treino: 713
Valid: 178
Acc: 0.7415730337078652

Treino: 713
Valid: 178
Acc: 0.8202247191011236

Rep: 4
Treino

In [29]:
len(resultados)


50

In [30]:
np.mean(resultados)


0.7692593057560732

<h1>Criar Submissão</h1>


In [32]:
sub = pd.Series(p, index=teste['PassengerId'], name='Survived')
sub.shape

ValueError: Length of passed values is 178, index implies 418.